# Snowpark Connection

> Helps Manage Snowflake Connection

In [1]:
#| default_exp connection

In [2]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
from pathlib import Path
from typing import Optional, Dict, Any, Tuple
from pydantic import BaseModel, field_validator
from snowflake.snowpark import Session
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import yaml
from logging import getLogger
import warnings
# Suppress the specific Pydantic warning about schema
warnings.filterwarnings("ignore", message="Field name \"schema\" .* shadows an attribute in parent \"BaseModel\"")

logger = getLogger(__name__)

In [ ]:
#| export

class AuthenticationError(Exception):
    """Custom exception for authentication errors."""
    pass

class SnowflakeConnection(BaseModel):
    """
    Unified Snowflake connection configuration and session management.
    Supports password, key-pair, and external authenticator methods with session caching.
    """
    account: str
    user: str
    password: Optional[str] = None
    private_key_path: Optional[Path] = None
    private_key_pem: Optional[str] = None
    authenticator: Optional[str] = None
    role: str = "DATASCIENTIST"
    warehouse: str = "DS_WH_XS"
    database: Optional[str] = None
    schema: Optional[str] = None

    # Session cache keyed by (role, warehouse, database)
    _session_cache: Dict[Tuple[str, str, Optional[str]], Session] = {}

    @field_validator("private_key_path", mode="before")
    def ensure_path_object(cls, v):
        return Path(v) if v and not isinstance(v, Path) else v

    @classmethod
    def from_yaml(cls, yaml_file: str = 'snowflake_config.yaml') -> 'SnowflakeConnection':
        """Create configuration from YAML file with environment variable fallback."""
        config = {}
        
        # Try loading from YAML first
        if os.path.isfile(yaml_file):
            with open(yaml_file, 'r') as file:
                config = yaml.safe_load(file).get('snowflake', {})

        # Environment variables override YAML
        env_vars = {
            'SNOWFLAKE_ACCOUNT': 'account',
            'SNOWFLAKE_USER': 'user',
            'SNOWFLAKE_PASSWORD': 'password',
            'SNOWFLAKE_ROLE': 'role',
            'SNOWFLAKE_WAREHOUSE': 'warehouse',
            'SNOWFLAKE_DATABASE': 'database',
            'SNOWFLAKE_SCHEMA': 'schema',
            'SNOWFLAKE_PRIVATE_KEY_PATH': 'private_key_path',
            'SNOWFLAKE_AUTHENTICATOR': 'authenticator'
        }

        for env_var, config_key in env_vars.items():
            if value := os.getenv(env_var):
                config[config_key] = value

        return cls(**config)

    def _load_private_key(self) -> bytes:
        """Load and format private key from file or PEM string."""
        try:
            if self.private_key_pem:
                key_data = self.private_key_pem.encode()
            elif self.private_key_path:
                with open(self.private_key_path, "rb") as key_file:
                    key_data = key_file.read()
            else:
                raise AuthenticationError("No private key source available")

            p_key = serialization.load_pem_private_key(
                key_data,
                password=None,
                backend=default_backend()
            )
            return p_key.private_bytes(
                encoding=serialization.Encoding.DER,
                format=serialization.PrivateFormat.PKCS8,
                encryption_algorithm=serialization.NoEncryption()
            )
        except Exception as e:
            raise AuthenticationError(f"Error loading private key: {str(e)}")

    def get_connection_params(
        self,
        role: Optional[str] = None,
        warehouse: Optional[str] = None,
        database: Optional[str] = None,
        schema: Optional[str] = None
    ) -> Dict[str, Any]:
        """Get connection parameters with optional overrides."""
        params = {
            "account": self.account,
            "user": self.user,
            "role": role or self.role,
            "warehouse": warehouse or self.warehouse,
        }

        # Add optional parameters if provided
        if database or self.database:
            params["database"] = database or self.database
        if schema or self.schema:
            params["schema"] = schema or self.schema

        # Handle authentication methods in priority order:
        # 1. External authenticator (e.g., OAuth)
        # 2. Key pair authentication
        # 3. Password authentication
        if self.authenticator:
            params["authenticator"] = self.authenticator
        elif self.private_key_pem or self.private_key_path:
            params["private_key"] = self._load_private_key()
        elif self.password:
            params["password"] = self.password
        else:
            raise AuthenticationError(
                "No authentication method provided. Please provide either "
                "authenticator, private_key, or password."
            )

        return params

    def get_session(
        self,
        role: Optional[str] = None,
        warehouse: Optional[str] = None,
        database: Optional[str] = None,
        schema: Optional[str] = None,
        use_cache: bool = True
    ) -> Session:
        """
        Get or create a Snowflake session.
        
        Args:
            role: Override default role
            warehouse: Override default warehouse
            database: Override default database
            schema: Override default schema
            use_cache: Whether to use session caching (default True)
            
        Returns:
            Session: A Snowflake session object
        """
        final_role = role or self.role
        final_warehouse = warehouse or self.warehouse
        final_database = database or self.database
        
        # Try to get cached session if caching is enabled
        if use_cache:
            cache_key = (final_role, final_warehouse, final_database)
            if cache_key in self._session_cache:
                session = self._session_cache[cache_key]
                if schema:  # Update schema if needed
                    session.use_schema(schema)
                return session

        # Create new session
        params = self.get_connection_params(final_role, final_warehouse, final_database, schema)
        try:
            session = Session.builder.configs(params).create()
            
            # Cache if enabled
            if use_cache:
                cache_key = (final_role, final_warehouse, final_database)
                self._session_cache[cache_key] = session
                logger.info(f"Created and cached new session for {cache_key}")
            
            return session
            
        except Exception as e:
            raise AuthenticationError(f"Failed to create session: {str(e)}")

    def close_all_sessions(self) -> None:
        """Close all cached sessions."""
        for session in self._session_cache.values():
            try:
                session.close()
            except Exception as e:
                logger.warning(f"Error closing session: {e}")
        self._session_cache.clear()

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close_all_sessions()

/Users/jdemlow/miniconda3/envs/custom_functions/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "SnowflakeConnection" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [10]:
#| skip

# 1. Username and Password
def username_password_example():
    """
    Example of using SnowflakeConnection with username/password authentication.
    """
    config = {
        'user': os.getenv('SNOWFLAKE_USER', ''),
        'password': os.getenv('SNOWFLAKE_PASSWORD', ''),
        'account': os.getenv('SNOWFLAKE_ACCOUNT', ''),
        'database': 'DATASCIENCE',
        'warehouse': 'DS_WH_XS',
        'schema': 'DEV',
        'role': 'DATA_SCIENTIST'
    }
    
    try:
        # Create connection using the config
        with SnowflakeConnection(**config) as conn:
            # Get session with specified schema
            session = conn.get_session()
            
            # Run test queries
            warehouse_info = session.sql("""
                SELECT 
                    CURRENT_WAREHOUSE(),
                    CURRENT_DATABASE(),
                    CURRENT_SCHEMA()
            """).collect()
            print("Connection successful!")
            print(f"Current settings: {warehouse_info}")
            
    except AuthenticationError as e:
        logger.error(f"Authentication failed: {e}")
        raise
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        raise


username_password_example()

Connection successful!
Current settings: [Row(CURRENT_WAREHOUSE()='DS_WH_XS', CURRENT_DATABASE()='DATASCIENCE', CURRENT_SCHEMA()=None)]


Example Setup Key Pair Authentication Setup

1. Generate a private key (run these commands in your local terminal, not in Snowflake)

```bash
openssl genrsa 2048 | openssl pkcs8 -topk8 -inform PEM -out rsa_key.p8 -nocrypt
# OR
openssl genrsa 2048 | openssl pkcs8 -topk8 -v2 des3 -inform PEM -out rsa_key.p8
openssl rsa -in rsa_key.p8 -pubout -out rsa_key.pub
```

2. Assign the public key to a Snowflake user
-- Replace 'YOUR_USERNAME' with the actual username

```sql
ALTER USER YOUR_USERNAME SET RSA_PUBLIC_KEY='<paste_public_key_here>';
```

3. Verify the key is set correctly

```sql
DESC USER YOUR_USERNAME;
```


In [11]:
#| skip
# Example usage
def key_pair_example():
    """
    Example of using SnowflakeConnection with key pair authentication.
    """
    # Convert key file path to proper Path object
    key_path = Path('~/keys/snowflake/rsa_key.p8').expanduser().resolve()
    
    config = {
        'user': os.getenv('SNOWFLAKE_USER', ''),
        'account': os.getenv('SNOWFLAKE_ACCOUNT', ''),
        'private_key_path': key_path,
        'database': 'DATASCIENCE',
        'warehouse': 'DS_WH_XS',
        'schema': 'DEV',
        'role': 'CORTEX_USER_ROLE'
    }
    
    try:
        # Create connection using the config
        with SnowflakeConnection(**config) as conn:
            session = conn.get_session()
            user_info = session.sql("""
                SELECT 
                    CURRENT_USER(),
                    CURRENT_ROLE()
            """).collect()
            print("Connection successful!")
            print(f"User info: {user_info}")
            
            # Test session caching
            cached_session = conn.get_session()
            print("Successfully retrieved cached session")
            
    except AuthenticationError as e:
        logger.error(f"Authentication failed: {e}")
        raise
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}")
        raise

key_pair_example()

Connection successful!
User info: [Row(CURRENT_USER()='JD_SERVICE_ACCOUNT_ADMIN', CURRENT_ROLE()='CORTEX_USER_ROLE')]
Successfully retrieved cached session


In [12]:
#| hide
import nbdev; nbdev.nbdev_export()